In [2]:
import warnings
import awkward as ak
import numpy as np
import hist as hs
from hist.intervals import ratio_uncertainty

from coffea import processor
from coffea.nanoevents.methods import vector, candidate
from numba import jit

from coffea.nanoevents import BaseSchema
import mplhep as hep
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from datetime import date
today = date.today()

plt.style.use(hep.style.CMS)
#plt.style.use('dark_background')

In [42]:
class LLP_ntuple_processor(processor.ProcessorABC):
    
    def delta_cls_gLLP(self, events):
        clusts = ak.zip(
            {
                'pt': ak.zeros_like(events.dtRechitClusterPhi),
                'phi': events.dtRechitClusterPhi,
                'eta': events.dtRechitClusterEta,
                'E': ak.zeros_like(events.dtRechitClusterPhi),
            },
            with_name = 'PtEtaPhiMLorentzVector',
            behavior = vector.behavior,                 
          )
        gLLPs = ak.zip(
            {
                'pt': ak.flatten(events.gLLP_pt, axis=None),
                'phi': ak.flatten(events.gLLP_phi, axis=None),
                'eta': ak.flatten(events.gLLP_eta, axis=None),
                'E': ak.flatten(events.gLLP_e, axis=None),
            },
            with_name = 'PtEtaPhiMLorentzVector',
            behavior = vector.behavior,                 
          )        
        cls_llp_pairs = ak.cartesian({
            "cls": clusts, 
            "llp": gLLPs
        }, nested = True) 
        
        deltaR = (cls_llp_pairs.cls).delta_r(cls_llp_pairs.llp)
        deltaEta = abs(cls_llp_pairs.cls.eta - cls_llp_pairs.llp.eta)
        deltaPhi = np.arctan2(np.sin(cls_llp_pairs.cls.phi - cls_llp_pairs.llp.phi), 
                              np.cos(cls_llp_pairs.cls.phi - cls_llp_pairs.llp.phi))

        deltaR, deltaEta, deltaPhi = ak.flatten(deltaR, axis=2), ak.flatten(deltaEta, axis=2), ak.flatten(deltaPhi, axis=2)
        return deltaR, deltaEta, deltaPhi
    
    def delta_cls_leadmuon(self, events):
        clusts = ak.zip(
            {
                'pt': ak.zeros_like(events.dtRechitClusterPhi),
                'phi': events.dtRechitClusterPhi,
                'eta': events.dtRechitClusterEta,
                'E': ak.zeros_like(events.dtRechitClusterPhi),
            },
            with_name = 'PtEtaPhiMLorentzVector',
            behavior = vector.behavior,                 
          )
        leadMuons = ak.zip(
            {
                'pt': events.leadMuonPt,
                'phi': events.leadMuonPhi,
                'eta': events.leadMuonEta,
                'E': events.leadMuonE,
            },
            with_name = 'PtEtaPhiMLorentzVector',
            behavior = vector.behavior,                 
          ) 
        cls_muon_pairs = ak.cartesian({
            "cls": clusts, 
            "muon": leadMuons
        }, nested = True)
 
        deltaR = (cls_muon_pairs.cls).delta_r(cls_muon_pairs.muon)
        deltaEta = abs(cls_muon_pairs.cls.eta - cls_muon_pairs.muon.eta)
        deltaPhi = np.arctan2(np.sin(cls_muon_pairs.cls.phi - cls_muon_pairs.muon.phi), 
                              np.cos(cls_muon_pairs.cls.phi - cls_muon_pairs.muon.phi))
        
        deltaR, deltaEta, deltaPhi = ak.flatten(deltaR, axis=2), ak.flatten(deltaEta, axis=2), ak.flatten(deltaPhi, axis=2)
        
        # count = 0
        # for r, e, p in zip(deltaR ,deltaEta, deltaPhi):
        #     if len(r) > 0 or len(e) > 0 or len(p)>0:
        #         print(r, np.sqrt(e*e + p*p))
        #         print()
        #         # print(r[0] == np.sqrt(e*e + p*p)[0])
        #         # if r[0] == np.sqrt(e*e + p*p)[0]:
        #         #     count += 1
        # print(count)
        
        return deltaR, deltaEta, deltaPhi
        
    
    def process(self, events):
        
        signame = 'Phi'
        
        # >>> output dict init >>>
        dataset = events.metadata['dataset']
        out = {
            dataset: {},
            f'{dataset}_cuts': {},
            f'{dataset}_vars': {},
        }
        # <<< output dict init <<<
        
        out[dataset][f'numEvents_pretrigger'] = len(events.lepPt)
        
        # >>> some preprocessing >>>
        muoncut = abs(events.lepPdgId) == 13
        events['muonE'] = events.lepE[muoncut]
        events['muonPt'] = events.lepPt[muoncut]
        events['muonEta'] = events.lepEta[muoncut]
        events['muonPhi'] = events.lepPhi[muoncut]
        events['muonPdgId'] = events.lepPdgId[muoncut]
        events['muonDZ'] = events.lepDZ[muoncut]
        events['muonLooseId'] = events.lepLooseId[muoncut]
        events['muonTightId'] = events.lepTightId[muoncut] 
        # events['muonType'] = events.lepMuonType[muoncut]
        # events['muonQuality'] = events.lepMuonQuality[muoncut]
        # events['muon_passHLTFilter'] = events.lepMuon_passHLTFilter[muoncut]
        events = events[ak.count(events.muonPt, axis=1) > 0] #kill all events with empty muons
        out[dataset][f'numEvents_|muonId| == 13'] = len(events.muonPt)
        #out[dataset][f'numEvents_|muonId| == 13'] = sum(ak.any(events.muonPt, axis=1))
        
        muoncut = abs(events.muonEta) < 1.5
        events['muonE'] = events.muonE[muoncut]
        events['muonPt'] = events.muonPt[muoncut]
        events['muonEta'] = events.muonEta[muoncut]
        events['muonPhi'] = events.muonPhi[muoncut]
        events['muonPdgId'] = events.muonPdgId[muoncut]
        events['muonDZ'] = events.muonDZ[muoncut]
        events['muonLooseId'] = events.muonLooseId[muoncut]
        events['muonTightId'] = events.muonTightId[muoncut]
        # events['muonType'] = events.muonType[muoncut]
        # events['muonQuality'] = events.muonQuality[muoncut]
        # events['muon_passHLTFilter'] = events.muon_passHLTFilter[muoncut]
        events = events[ak.count(events.muonPt, axis=1) > 0] #kill all events with empty muons, not sure if necessary
        out[dataset][f'numEvents_|muonEta| < 1.5'] = len(events.muonPt)
        #out[dataset][f'numEvents_|muonEta| < 1.5'] = sum(ak.any(events.muonPt, axis=1))

        # muoncut = ak.any(events.muon_passHLTFilter[:,:,range(60,68)], axis=2)
        # events['muonE'] = events.muonE[muoncut]
        # events['muonPt'] = events.muonPt[muoncut]
        # events['muonEta'] = events.muonEta[muoncut]
        # events['muonPhi'] = events.muonPhi[muoncut]
        # events['muonPdgId'] = events.muonPdgId[muoncut]
        # events['muonDZ'] = events.muonDZ[muoncut]
        # events['muonLooseId'] = events.muonLooseId[muoncut]
        # events['muonTightId'] = events.muonTightId[muoncut]
        # events['muonType'] = events.muonType[muoncut]
        # events['muonQuality'] = events.muonQuality[muoncut]
        # events['muon_passHLTFilter'] = events.muon_passHLTFilter[muoncut]  
        # events = events[ak.count(events.muonPt, axis=1) > 0] #kill all events with empty muons, not sure if necessary
        # out[dataset][f'numEvents_muonHLTReq'] = len(events.muonPt)

        # muoncut = events.muonQuality > 2**25
        # events['muonE'] = events.muonE[muoncut]
        # events['muonPt'] = events.muonPt[muoncut]
        # events['muonEta'] = events.muonEta[muoncut]
        # events['muonPhi'] = events.muonPhi[muoncut]
        # events['muonPdgId'] = events.muonPdgId[muoncut]
        # events['muonDZ'] = events.muonDZ[muoncut]
        # events['muonLooseId'] = events.muonLooseId[muoncut]
        # events['muonTightId'] = events.muonTightId[muoncut]
        # events['muonType'] = events.muonType[muoncut]
        # events['muonQuality'] = events.muonQuality[muoncut]
        # events['muon_passHLTFilter'] = events.muon_passHLTFilter[muoncut] 
        # events = events[ak.count(events.muonPt, axis=1) > 0] #kill all events with empty muons, not sure if necessary
        # out[dataset][f'numEvents_muonQual > 2^25'] = len(events.muonPt) 
        
        leadcut = (ak.max(events.muonPt, axis=1, mask_identity=True) == events.muonPt)
        events['leadMuonE'] = events.muonE[leadcut][:,0]
        events['leadMuonPt'] = events.muonPt[leadcut][:,0]
        events['leadMuonEta'] = events.muonEta[leadcut][:,0]
        events['leadMuonPhi'] = events.muonPhi[leadcut][:,0]
        events['leadMuonPdgId'] = events.muonPdgId[leadcut][:,0]
        events['leadMuonDZ'] = events.muonDZ[leadcut][:,0]
        events['leadMuonLooseId'] = events.muonLooseId[leadcut][:,0]
        events['leadMuonTightId'] = events.muonTightId[leadcut][:,0]
        # events['leadMuonType'] = events.muonType[leadcut][:,0]
        # events['leadMuonQuality'] = events.muonQuality[leadcut][:,0]
        # events['leadMuon_passHLTFilter'] = events.muon_passHLTFilter[leadcut][:,0]    
        out[dataset][f'numEvents_muonLeadPtReq'] = len(events.leadMuonPt) #this will of course be the same as the last cut
        
        if signame in dataset:
            cls_llp_deltaR, cls_llp_deltaEta, cls_llp_deltaPhi = self.delta_cls_gLLP(events)
            events['dtRechitCluster_llp_deltaR'] = cls_llp_deltaR
            events['dtRechitCluster_llp_deltaEta'] = cls_llp_deltaEta
            events['dtRechitCluster_llp_deltaPhi'] = cls_llp_deltaPhi

        cls_leadmuon_deltaR, cls_leadmuon_deltaEta, cls_leadmuon_deltaPhi = self.delta_cls_leadmuon(events)
        events['dtRechitCluster_leadmuon_deltaR'] = cls_leadmuon_deltaR
        events['dtRechitCluster_leadmuon_deltaEta'] = cls_leadmuon_deltaEta
        events['dtRechitCluster_leadmuon_deltaPhi'] = cls_leadmuon_deltaPhi
        
        # HLT_MU_IP_sel = range(1157,1197)
        # eventHLTcut = ak.any(events.HLTDecision[:,HLT_MU_IP_sel], axis=1)
        # events = events[eventHLTcut]
        # <<< some preprocessing <<<


        # >>> cut definitions >>>        
        dtCuts = {'eventHLT_cut': ak.values_astype(ak.ones_like(events.nDtRechitClusters), 'bool')} #dummy truth vector of same shape as dt variables
        

        if signame in dataset:
            dtCuts['llp_acc'] = (events.gLLP_dt == 1)
                    
        dtCuts['in_dt'] = (events.nDtRechitClusters >= 1)  

        dtCuts['dR_llp_cls < .4'] = (events.dtRechitCluster_llp_deltaR < .4) 
        #print(dtCuts['dR_llp_cls < .4'].layout.minmax_depth)
        dtCuts['dR_lmuon_cls > .8'] = (events.dtRechitCluster_leadmuon_deltaR > .8) 
        #print(dtCuts['dR_lmuon_cls > .8'].layout.minmax_depth)

        
#         dtCuts['ME11_veto'] = ((events.dtRechitClusterNRechitChamberPlus11 <= 0)&
#                              (events.dtRechitClusterNRechitChamberMinus11 <= 0)) 
        
#         dtCuts['ME12_veto'] = ((events.dtRechitClusterNRechitChamberPlus12 <= 0)&
#                              (events.dtRechitClusterNRechitChamberMinus12 <= 0)) 
        
#         dtCuts['re12_cut'] = (events.dtRechitCluster_match_RE12_0p4 == 0)
#         dtCuts['mb1_cut'] = (events.dtRechitCluster_match_MB1Seg_0p4 == 0)
#         dtCuts['rb1_cut'] = (events.dtRechitCluster_match_RB1_0p4 == 0)
#         dtCuts['muon_veto_pt < 20'] = (events.dtRechitClusterMuonVetoPt < 20)
        
#         dtCuts['-5 < cls_time < 12.5'] = ((events.dtRechitClusterTimeWeighted <= 12.5)&
#                                    (events.dtRechitClusterTimeWeighted >= -5  ))
        
#         dtCuts['|cls_timeSpread| < 20'] = (events.dtRechitClusterTimeSpreadWeightedAll <= 20)
#         dtCuts['|cls_eta| < 1.9'] = (abs(events.dtRechitClusterEta) < 1.9)
#         dtCuts['cls_size > 130'] = (events.dtRechitClusterSize > 130)
        
#         dtCuts['cut_based_ID'] = (((events.dtRechitClusterNStation10 >  1) & (abs(events.dtRechitClusterEta) < 1.9)) |
#            ((events.dtRechitClusterNStation10 == 1) & (abs(events.dtRechitClusterAvgStation10) == 4) & (abs(events.dtRechitClusterEta) < 1.8)) |
#            ((events.dtRechitClusterNStation10 == 1) & (abs(events.dtRechitClusterAvgStation10) == 3) & (abs(events.dtRechitClusterEta) < 1.6)) |
#            ((events.dtRechitClusterNStation10 == 1) & (abs(events.dtRechitClusterAvgStation10) == 2) & (abs(events.dtRechitClusterEta) < 1.6)) |
#            ((events.dtRechitClusterNStation10 == 1) & (abs(events.dtRechitClusterAvgStation10) == 1) & (abs(events.dtRechitClusterEta) < 1.1)))
        # <<< cut definitions <<<

        # >>> variables to be plotted >>>                
        __ = lambda x: x
        bins = 30
        
        #must be same shape as any dt variable
#         dtVars = {
#             'dtRechitClusterNRechitChamberPlus11':  [bins,    0,  10, __, ],
#             'dtRechitClusterNRechitChamberMinus11': [bins,    0,  10, __, ],
#             'dtRechitClusterNRechitChamberPlus12':  [bins,    0,  10, __, ],
#             'dtRechitClusterNRechitChamberMinus12': [bins,    0,  10, __, ],
#             'dtRechitCluster_match_RE12_0p4':       [bins,    0,  10, __, ],
#             'dtRechitCluster_match_MB1Seg_0p4':     [bins,    0,  10, __, ],
#             'dtRechitCluster_match_RB1_0p4':        [bins,    0,  10, __, ],
#             'dtRechitClusterMuonVetoPt':            [bins,    0, 100, __, ],
#             'dtRechitClusterTimeWeighted':          [bins,  -20,  20, __, ],
#             'dtRechitClusterTimeSpreadWeightedAll': [bins,    0,  30, __, ],
#             'dtRechitClusterEta':                   [bins,    0,   3, abs,],
#             'dtRechitClusterSize':                  [bins,    0, 250, __, ],
#             'dtRechitClusterNStation10':            [bins,    0,   5, __, ],
#             'dtRechitClusterAvgStation10':          [bins,    0,   5, abs, ],
#         }

#         if signame in dataset:
#             dtVars['dtRechitCluster_llp_deltaR']   = [bins, 0, 5, __,]
#             dtVars['dtRechitCluster_llp_deltaEta'] = [bins, 0, 5, abs,]
#             dtVars['dtRechitCluster_llp_deltaPhi'] = [bins, 0, 5, __,]

#         dtVars['dtRechitCluster_leadmuon_deltaR']   = [bins, 0, 5, __,]
#         dtVars['dtRechitCluster_leadmuon_deltaEta'] = [bins, 0, 5, abs,]
#         dtVars['dtRechitCluster_leadmuon_deltaPhi'] = [bins, 0, 5, __,]
        
#         #must be flat variables of length nEvents
#         eventVars = {
#             #'metEENoise': [bins,   0, 100, __, ],
#         }
        # <<< variables to be plotted <<<


        # >>> create hists >>> 
        bigCut = dtCuts[list(dtCuts.keys())[0]] #sets first cut
        for cut in dtCuts:
            out[f'{dataset}_cuts'][cut] = 1       
            bigCut = bigCut & dtCuts[cut]
            
            
            if bigCut.layout.minmax_depth == (2,2):
                temp = ak.any(bigCut, axis=1)
            elif bigCut.layout.minmax_depth == (1,1):
                temp = bigCut
                            
            out[dataset][f'numEvents_{cut}'] = sum(temp)
            #out[dataset][f'reweights_{cut}'] = sum()
            
            # for var in dtVars:
            #     out[f'{dataset}_vars'][var] = 1  
            #     v = dtVars[var]
            #     data = ak.flatten(events[var][bigCut], axis=None)
            #     data = v[3](data)
            #     key = f'{var} with {cut}'
            #     out[dataset][key] = hs.Hist.new.Reg(v[0], v[1], v[2], name=var, label=var).Double()
            #     out[dataset][key].fill(data) 
        # <<< create hists <<<
        
        
        return out
    
    def postprocess(self, accumulator):
        return accumulator
    

In [43]:
prefix = 'root://cmsxrootd.fnal.gov//store/user/christiw/displacedJetMuonAnalyzer/bparking/V1p19/MC_Fall18/v1/v5/normalized/'
fileset = {'Phi': ['root://cmsxrootd.fnal.gov//store/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/V1p17/MC_Fall18/v2/v3/normalized/BToKPhi_MuonGenFilter_mPhi1p0_ctau1000_1pb_weighted.root',]}
# fileset = {
#             'PhiToPi0Pi0_mPhi0p3_ctau1000':
#                 [prefix + 'BToKPhi_MuonGenFilter_PhiToPi0Pi0_mPhi0p3_ctau1000_1pb_weighted.root'],

#             'PhiToPi0Pi0_mPhi0p3_ctau300':
#                 [prefix + '/BToKPhi_MuonGenFilter_PhiToPi0Pi0_mPhi0p3_ctau300_1pb_weighted.root'],

#             'PhiToPiPlusPiMinus_mPhi0p3_ctau300':
#                 [prefix + 'BToKPhi_MuonGenFilter_PhiToPiPlusPiMinus_mPhi0p3_ctau300_1pb_weighted.root'],

#             'PhiToPiPlusPiMinus_mPhi0p3_ctau1000':
#                 [prefix + 'BToKPhi_MuonGenFilter_PhiToPiPlusPiMinus_mPhi0p3_ctau1000_1pb_weighted.root'],
#             }
#fileset['backgroundNew'] = ['root://cmsxrootd.fnal.gov//store/user/christiw/displacedJetMuonAnalyzer/bparking/V1p19/Data2018_UL/v6/normalized/ParkingBPH4_2018A_goodLumi.root']

out = processor.run_uproot_job(
    fileset,
    treename="MuonSystem",
    processor_instance=LLP_ntuple_processor(),
    executor=processor.futures_executor,
    #executor_args={"schema": BaseSchema, "workers": 16},
    executor_args={"schema": BaseSchema, "workers": 1},
    maxchunks = 1
    #chunksize=100_000_000
)

Output()

(2, 2)
(2, 2)


In [38]:
sigcuts = list(out[f'{list(fileset.keys())[0]}_cuts'].keys())
#bkgcuts = list(out['backgroundNew_cuts'].keys())

sigvars = list(out[f'{list(fileset.keys())[0]}_vars'].keys())
# bkgvars = list(out['backgroundNew_vars'].keys())

In [39]:
datasets = list(fileset.keys())

for dataset in datasets:
    print(dataset)
    for key in out[dataset]:
        if key.startswith('numEvents_'):# and ('NStation10' not in key):
            print(f"{key[10:]}, {out[dataset][key]}")
            #print(f"{out[dataset][key]}")
    print()

Phi
pretrigger, 823789
|muonId| == 13, 812236
|muonEta| < 1.5, 551367
muonLeadPtReq, 551367
eventHLT_cut, 551367
llp_acc, 85727
in_dt, 41433
dR_llp_cls < .4, 12867
dR_lmuon_cls > .8, 5256

